# IESB - Miner II - Trabalho Final
* Objetivo: elaborar modelo preditivo em Python para prever o nível de pobreza de chefes de família da Costa Rica, a partir de base de dados disponibilizada no Kaggle
* Conteúdo:
* * Análise exploratória dos dados
* * Pré-processamento dos dados
* * Aplicação de técnicas de tratamento de classes desbalanceadas 
* * Avaliação dos melhores algoritmos de classificação, usando base de validação
* * Aplicação dos melhores algoritmos na base de teste
* * Submissão dos resultados ao Kaggle

# Bibliotecas

In [ ]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import resample

import imblearn
from imblearn.over_sampling import SMOTE

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Carregando os dados

In [ ]:
df_train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

In [ ]:
# Juntando dados de treino e teste
df_all = df_train.append(df_test)
df_all.info()

## Considerações iniciais
- Cada linha representa uma pessoa
- Múltiplas pessoas podem fazer parte de uma única residência
- Mais de uma família pode viver na mesma casa
- A previsão deve se dar apenas para os chefes de família 

# Análise Exploratória / Pré-processamento dos dados

In [ ]:
# Tamanhos das bases de treino e teste
# O número de dados de treino é menor que os dados de teste
# Em tese, essa característica torna o trabalho de classificação mais difícil
df_train.shape, df_test.shape

In [ ]:
## Informações do dataset 
# 9 variáveis float, 129 inteiras e 5 object
df_all.info()

## Tratamento das Variáveis Object

In [ ]:
# Variáveis do tipo 'object': Id, idhogar, dependency, edjefe, edjefa
# Id e idhogar são identificadores de pessoa e casa - ok
# dependency, edjefe, edjefa requerem tratamento
df_all.select_dtypes('object').head(5)

### Variáveis edjefe e edjefa
* Anos de escolaridade do chefe / da chefe de família 
* Do dicionário de dados, yes=1 e no=0 => basta fazer a substituição

In [ ]:
# Substituindo 'yes' e 'no'
yes_no_map = {'no': 0, 'yes': 1}

df_all['edjefe'] = df_all['edjefe'].replace(yes_no_map).astype(np.float32)
df_all['edjefa'] = df_all['edjefa'].replace(yes_no_map).astype(np.float32)


### Variável dependency
* Taxa de dependência: (membros da família  potencialmente dependentes) / (número de trabalahadores potencialmente ativos)
* Fórmula: (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)

In [ ]:
df_all['dependency'].value_counts()

In [ ]:
# Os valores 0 e 1 não estão na lista acima, confirmando que a regra 'yes'=1 e 'no'=0 se aplica
# É feita a substituição:
df_all['dependency'] = df_all['dependency'].replace(yes_no_map).astype(np.float32)

In [ ]:
# Confirmando os resultados
# => não existem outras variáveis do tipo object após os tratamentos, às exceções dos identificadores de pessoa (Id) e casa (idhogar)
df_all.select_dtypes('object').head(5)

# Tratamento dos Valores Ausentes 

In [ ]:
# Do resultado, as variáveis com valores nulos são: rez_esc, v18q1, v2a1, SQBmeaned, meaneduc
# os valores nulos da variável Target se referem aos dados de teste => comportamento esperado
df_all.isnull().sum().sort_values(ascending=False).head(7)

In [ ]:
# Uma visualização mais sofisticada....

df_all_null = df_all.isnull().sum()

# filtra apenas as variáveis que contêm valores nulos, representadas com relação ao total
df_all_null_non_zero = (df_all_null[df_all_null>0] / df_all.shape[0]).sort_values(ascending=False)

sns.barplot(x=df_all_null_non_zero, y=df_all_null_non_zero.index)
_ = plt.title('Percentual de Valores Nulos')

### Variável rez_esc
* Anos atrasados na escola

In [ ]:
# O valor zero (0) é presente no dados => não é possível aplicar uma regra de negócio específica
df_all['rez_esc'].describe()

In [ ]:
# Investigação do outlier 99 (valor máximo verificado no describe)
df_all['rez_esc'].value_counts()

In [ ]:
# Apenas uma ocorrência deste outlier, provavelmente um erro de alimentação de dados
# este será substituído pelo pior valor de anos atrasados presente na base (5)
df_all['rez_esc'].replace(to_replace=99, value=5, inplace=True)
df_all['rez_esc'].value_counts()

In [ ]:
# solução: atribuir o valor -1 para os valores nulos de rez_esc
# isso é uma estratégia válida para algoritmos baseados em árvore
df_all['rez_esc'].fillna(-1, inplace=True)

### Variável v2a1
* Valor do aluguel

In [ ]:
# O valor zero (0) é presente no dados => não é possível aplicar uma regra de negócio específica
df_train['v2a1'].describe()

In [ ]:
# solução: atribuir um valor -1 para os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

### Variável v18q1
* Número de tablets que a família possui

In [ ]:
# correlacionada com  a variável 'v18q', que identifica se a família possui um tablet
# => valores nulos correspondem a zero tablet
df_all['v18q1'].fillna(0, inplace=True)

### Variável meaneduc
* Média de anos de educação para adultos (+18)

In [ ]:
# se referem a uma fração muito pequena do dataset (0,1%)
df_all['meaneduc'].isnull().sum() / df_all['meaneduc'].count() * 100

In [ ]:
# imputação de 'meaneduc' pela mediana
df_all['meaneduc'].fillna(df_all['meaneduc'].median(), inplace=True)

In [ ]:
## SQBmeaned: derivado de 'meaneduc'
# imputação pela mediana
df_all['SQBmeaned'].fillna(df_all['SQBmeaned'].median(), inplace=True)

In [ ]:
## Verificação do dataset após os tratamentos dos missing values
# pelo resultado, percebe-se que as variáveis com valores ausentes foram devidamente tratadas
df_all.isnull().sum().sort_values(ascending=False).head(7)

# Distribuição da variável 'Target'

Valores possíveis:
* 1: pobreza extrema
* 2: pobreza moderada
* 3: famílias vulneráveis
* 4: famílias não vulneráveis

In [ ]:
# Confirmação que não existem  valores nulos em 'Target'
df_train['Target'].isnull().sum()

In [ ]:
# Contagem dos valores 'Target'
df_train['Target'].value_counts().sort_index(ascending=False)

In [ ]:
# Distribuição dos valores 'Target'
ax = sns.countplot(x='Target', data=df_train)
_ = plt.title('Distribuição dos valores Target')

Os resultados acima indicam um problema de classificação com classes desbalanceadas. A quantidade de famílias em situação não vulnerável é muito superior ao número das demais famílias

# Chefes de família
* Somente os chefes de família são usados na pontuação. 
* Todos os membros da família são incluídos no teste + o envio da amostra, mas apenas os chefes de família são pontuados.
* Variável parentesco1=1 indica se é chefe de família 


In [ ]:
# Número e percentual de chefes de família na base de treino
n_chefes_familia = df_train[df_train['parentesco1']==1]['parentesco1'].sum()
n_chefes_familia, n_chefes_familia / df_train.shape[0] * 100

In [ ]:
# Número e percentual de chefes de família na base de teste
n_chefes_familia = df_test[df_test['parentesco1']==1]['parentesco1'].sum()
n_chefes_familia, n_chefes_familia / df_test.shape[0] * 100

* 31.1% da base de treino é composta de chefes de família
* 30.7% da base de treino é composta de chefes de família
* Nota-se o balanceamento da distribuição entre as quantidades de chefes de família nas bases de treino e teste


# Exclusão de Variáveis
* Para redução da dimensionalidade, as variáveis que representam o quadrado de outras variáveis (SQBxxx) são descartadas
* Eles seriam úteis para um modelo linear, mas são inúteis para um modelo baseado em árvore e podem confundi-lo
* Fonte: https://www.kaggle.com/mlisovyi/feature-engineering-lighgbm-with-f1-macro

In [ ]:
# Exclusão variáveis SQBxxx
col_excluir = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned']
df_all.drop(col_excluir, axis=1, inplace=True)
df_train.drop(col_excluir, axis=1, inplace=True)
df_test.drop(col_excluir, axis=1, inplace=True)

In [ ]:
## Conferência final do dataframe
# sem valores ausentes; sem variáveis object
df_all.info()

# Análise de Correlação

In [ ]:
correlacao = df_train.corr()
correlacao = correlacao['Target'].sort_values(ascending=False)

print(f'10 features positivas mais relevantes: \n{correlacao.head(10)}')
print('\n', '=' * 50, '\n')
print(f'10 features negativas mais relevantes:: \n{correlacao.tail(10)}')

Considerações:
* As variáveis mais correlacionadas com taxa de pobreza estão relacionadas com a educação dos adultos (meaneduc, escolari), número de criancas na casa (hogar_nin, r4t1), se a casa tem teto (cielorazo). 
* Isso coincide com o senso comum

In [ ]:
# Observação: do resultado acima, chamou atenção o valor NaN da variável 'elimbasu5'
# Investigando-se a mesma nota, nota-se que ela possui apenas um valor para toda a base de treinamento, portanto sem nenhuma função explicativa
df_train['elimbasu5'].value_counts()

In [ ]:
# Decisão: eliminar essa variável
df_all.drop('elimbasu5', axis=1, inplace=True)
df_train.drop('elimbasu5', axis=1, inplace=True)
df_test.drop('elimbasu5', axis=1, inplace=True)

# Separação das bases de treino e teste

In [ ]:
# Separa as bases de treino e teste
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

train = df_all[~df_all['Target'].isnull()]
test = df_all[df_all['Target'].isnull()]

train.shape, test.shape

# Balanceamento das Classes

### Técnica SMOTE
* Não obteve resultados satisfatórios
* F1-Score: 0.8288343045151485
* Acurácia: 0.830938292476754

In [ ]:
# Aplica over-sampling SMOTE
# sm = SMOTE()
# X_train, y_train = sm.fit_resample(X_train,y_train)
# X_valid, y_valid = sm.fit_resample(X_valid, y_valid)

### Técnica de over-sampling tradicional (sklearn)

In [ ]:
# Separa os dados de treino de acordo com a classificação de pobreza (valor de Target)
train_1 = train[train['Target'] == 1]
train_2 = train[train['Target'] == 2]
train_3 = train[train['Target'] == 3]
train_4 = train[train['Target'] == 4]

train_1.shape, train_2.shape,train_3.shape,train_4.shape

In [ ]:
# Aplica over-sampling na base de treino
train_1_over = resample(train_1,                 # aumenta a classe menor
                       replace=True,             # sample com replacement
                       n_samples=len(train_4),   # iguala à maior classe (4)
                       random_state=42)

train_2_over = resample(train_2,               
                       replace=True,             
                       n_samples=len(train_4),  
                       random_state=42)

train_3_over = resample(train_3,               
                       replace=True,             
                       n_samples=len(train_4),  
                       random_state=42)


train_1_over.shape,train_2_over.shape,train_3_over.shape,train_4.shape

In [ ]:
# Junta os dados
train = pd.concat([train_1_over, train_2_over, train_3_over, train_4])
train.shape


# Treinamento Inicial - com dados de validação
* Utiliza base de validação para a escolha do melhor algoritmo
* Os dois melhores algoritmos selecionados nesta fase serão utilizado para treinar com a base completa de treino
* O melhos dos dois resultados será submetido à competição 

In [ ]:
# Separa a base de treino em treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

X_train = train[feats]
y_train = train['Target'].astype(int)

X_valid = valid[feats]
y_valid = valid['Target'].astype(int)

X_test = test[feats]

train.shape, valid.shape, test.shape

In [ ]:
# Função que avalia F1-score e acurácia do modelo
def avalia_modelo (val_true, val_pred):
    acuracia = accuracy_score(y_true=val_true, y_pred=val_pred)
    f1score = f1_score(y_true=val_true, y_pred=val_pred, average='macro')
    return f1score, acuracia

In [ ]:
## Random Forest 
rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, rf.predict(X_valid))

# Apresentação dos resultados
print ("Random Forest")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

In [ ]:
## Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, gbm.predict(X_valid))

# Apresentação dos resultados
print ("Gradient Boosting")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

In [ ]:
## XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
xgb.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, xgb.predict(X_valid))

# Apresentação dos resultados
print ("XGBoost")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

In [ ]:
## AdaBoost
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, abc.predict(X_valid))

# Apresentação dos resultados
print ("AdaBoost")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

In [ ]:
## CatBoost
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(random_state=42)
cbc.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, cbc.predict(X_valid))


# Apresentação dos resultados
print ("CatBoost")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

In [ ]:
## LightGBM
# parâmetros derivados de https://www.kaggle.com/denismarcio/improve-vision-with-lightgbm/
import lightgbm as lgb

clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.1, objective='multiclass',
                                 random_state=None, silent=True, metric='None', 
                                 n_jobs=4, n_estimators=5500, class_weight='balanced',
                                 colsample_bytree =  0.89, min_child_samples = 90, num_leaves = 56, subsample = 0.96)
clf.fit(X_train, y_train)

# Avalia modelo
f1score, acuracia = avalia_modelo(y_valid, clf.predict(X_valid))

# Apresentação dos resultados
print ("LightGBM")
print("F1-Score:", f1score)
print("Acurácia:", acuracia)

## Análise dos resultados - com base de validação
#### Random Forest:
    F1-Score: 0.9939934343153598
    Acurácia: 0.9940072954663888

#### LightGBM
    F1-Score: 0.9932042106373568
    Acurácia: 0.993225638353309

### CatBoost
    F1-Score: 0.976593475869052
    Acurácia: 0.9768108389786347

### Gradient Boosting
    F1-Score: 0.6199833610920772
    Acurácia: 0.6206357477853048

### XGBoost
    F1-Score: 0.939220952973292
    Acurácia: 0.9395518499218343

### AdaBoost
    F1-Score: 0.5263839922419775
    Acurácia: 0.52892131318395
* Dos resultados acima, os algoritmos com melhor desempenho foram o Random Forest e o LightGBM
* Estes dois algoritmos serão utilizados para o treino com toda a base de treino (sem validação) e depois aplicados na base de teste, para submissão ao Kaggle

# Treino dos melhores algorimos - toda a base de treino

In [ ]:
# Recupera as bases de treino e teste

feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

# Recupera a base de treino com oversampling 
train_full = pd.concat([train_1_over, train_2_over, train_3_over, train_4])
test = df_all[df_all['Target'].isnull()]

X_train_full = train_full[feats]
y_train_full = train_full['Target'].astype(int)

X_test = test[feats]

train_full.shape, test.shape

In [ ]:
## Random Forest 
rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)
rf.fit(X_train_full, y_train_full)


In [ ]:
## LightGBM
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.1, objective='multiclass',
                                 random_state=None, silent=True, metric='None', 
                                 n_jobs=4, n_estimators=5500, class_weight='balanced',
                                 colsample_bytree =  0.89, min_child_samples = 90, num_leaves = 56, subsample = 0.96)
clf.fit(X_train_full, y_train_full)


# Submissão para Competição

In [ ]:
## Realiza previsão do valor de Target na base de teste 

# Random Forest
#test['Target'] = rf.predict(X_test).astype(int)

# LightGBM
test['Target'] = clf.predict(X_test).astype(int)


In [ ]:
# Cria o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

# Resultados Finais

* LightGBM: 0.40331  
* Random Forest: 0.37312 
* Embora o Random Forest tenha obtido um resultado ligeiramente melhor na base de validação, o algoritmo LightGBM obteve melhor desempenho na base de teste, ou seja, soube generalizar melhor


# Sugestões de Trabalhos Futuros
* Melhorar a etapa de feature engineering, fazendo consolidação de variáveis 
* Aprimorar a análise relacionada aos chefes de família, dado que somente estes que são avaliados pelo Kaggle. Avaliar a possibilidade de criar mais colunas, trazendo informações gerais dos membros de família para o chefe de família